In [2]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

startTime = datetime.now()

# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
NB_EPOCH = 50
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
NUM_TO_AUGMENT=5
OPTIM = RMSprop()

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# augumenting
print("Augmenting training set images...")
datagen = ImageDataGenerator(rotation_range=40, 
                            width_shift_range=0.2, 
                            height_shift_range=0.2, 
                            zoom_range=0.2, 
                            horizontal_flip=True, 
                            fill_mode='nearest')


xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # (3, 32, 32)
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32)
    for x_aug in datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='cifar', save_format='jpeg'):
        if num_aug >= NUM_TO_AUGMENT:
            break
        xtas.append(x_aug[0])
        num_aug += 1

print("Augment completed in: " + str(datetime.now() - startTime))

#fit the dataget
datagen.fit(X_train)

# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

#AUGMENTATION MODEL TRAINING
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE), samples_per_epoch=X_train.shape[0], epochs=NB_EPOCH, verbose=VERBOSE)

#NON-AUGMENTATION MODEL TRAINING
#model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)

score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])
print("Total time: " + str(datetime.now() - startTime))

#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save_weights('cifar10_weights.h5', overwrite=True)

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Augmenting training set images...
Augment completed in: 0:07:02.797198


c:\Users\prest\Desktop\College_Programing\CS-370-Current_Emerging_Trends_in_CS\venv\lib\site-packages\ipykernel_launcher.py:82: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
c:\Users\prest\Desktop\College_Programing\CS-370-Current_Emerging_Trends_in_CS\venv\lib\site-packages\ipykernel_launcher.py:98: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, verbose=1, steps_per_epoch=390)`


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 64)       

I feel that an algorithm like the one we used for this assignment could easily be trained for things like facial recognition. In fact, facial recognition algorithms already exist in today’s day and age. While they can be used for good things such as locating criminals, human trafficking victims, etc. there is the fact that these types of algorithms can be used for some not so good things. Would you like it if your every move was being tracked and logged? Imagine if you were walking home one day and there were no cars on the road. You decide to cross the road while there are no cars rather than walk further up and wait at the crosswalk. This would be considered jaywalking and is a fine-able offence in many states. But no cars were present, so is it that big of a deal? It might be to you when you get a fine in the mail because a camera using a facial recognition algorithm identified you as you were committing the infraction. There’s actually a country where something like this has already been implemented, and that place is China. Alfred Ng from CNET writes “Cameras set up at crosswalks to identify and post photos of jaywalkers are commonplace, and a report from Abacus in May 2019 showed photos of children jaywalking on a digital billboard. The local traffic police said "children should be treated the same as adults," according to the outlet.” (Ng, 2020). This may not seem like that much of an issue as the whole reason jaywalking laws exist is to help keep pedestrians safe, using facial recognition software in this manner also requires extensive data collection. I would argue that most people would prefer it if their government didn’t track their every move. What if a government used the data collected to commit racism or persecute a minority population? Most people would agree that this is the exact opposite of what facial recognition software should be used for. Well, the Chinese government doesn’t agree. “…documents and interviews show that the authorities are also using a vast, secret system of advanced facial recognition technology to track and control the Uighurs, a largely Muslim minority. It is the first known example of a government intentionally using artificial intelligence for racial profiling, experts said.” (Mozur, 2019). This is a demonstration of what is probably the primary ethical/privacy concern when it comes to image/facial recognition algorithms. While they can be used for good, they can also very easily be used for bad. 

References

Mozur, P. (2019, April 14). One Month, 500,000 Face Scans: How China Is Using A.I. to Profile a Minority. Retrieved from The New York Times: https://www.nytimes.com/2019/04/14/technology/china-surveillance-artificial-intelligence-racial-profiling.html

Ng, A. (2020, August 11). How China uses facial recognition to control human behavior. Retrieved from CNET: https://www.cnet.com/news/politics/in-china-facial-recognition-public-shaming-and-control-go-hand-in-hand/